## Bayesian methods of hyperparameter optimization

In addition to the random search and the grid search methods for selecting optimal hyperparameters, we can use Bayesian methods of probabilities to select the optimal hyperparameters for an algorithm.

In this case study, we will be using the BayesianOptimization library to perform hyperparmater tuning. This library has very good documentation which you can find here: https://github.com/fmfn/BayesianOptimization

You will need to install the Bayesian optimization module. Running a cell with an exclamation point in the beginning of the command will run it as a shell command — please do this to install this module from our notebook in the cell below.

In [ ]:
#! pip install bayesian-optimization

CatBoost and LightGBM are both gradient boosting on decision trees.

 - [BayesianOptimization github](https://github.com/bayesian-optimization/BayesianOptimization)
   - see helpful notebooks linked on homepage, in /examples
 - [CatBoost](https://catboost.ai/en/docs/concepts/python-installation)
 - [LightGBM](https://github.com/microsoft/LightGBM)

In [1]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import lightgbm
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier, cv, Pool

## How does Bayesian optimization work?

Bayesian optimization works by constructing a posterior distribution of functions (Gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.

<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true" />
As you iterate over and over, the algorithm balances its needs of exploration and exploitation while taking into account what it knows about the target function. At each step, a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with an exploration strategy (such as UCB — aka Upper Confidence Bound), or EI (Expected Improvement). This process is used to determine the next point that should be explored (see the gif below).
<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bayesian_optimization.gif" />

## Let's look at a simple example

The first step is to create an optimizer. It uses two items:
* function to optimize
* bounds of parameters

The function is the procedure that counts metrics of our model quality. The important thing is that our optimization will maximize the value on function. Smaller metrics are best. Hint: don't forget to use negative metric values.

Here we define our simple function we want to optimize.

In [2]:
def simple_func(a, b):
    return a + b

Now, we define our bounds of the parameters to optimize, within the Bayesian optimizer.
 - inputs to Optimization objection
   - function (simple_func)
   - bounds for each dimension/parameter of the function
   - can define random state, set verbosity
 - Optimization object functions
   - **maximize**

In [6]:
optimizer = BayesianOptimization(
    simple_func,
    {'a': (1, 3),
    'b': (4, 7)},
    random_state=42)

These are the main parameters of this function:

* **n_iter:** This is how many steps of Bayesian optimization you want to perform. The more steps, the more likely you are to find a good maximum.

* **init_points:** This is how many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

Let's run an example where we use the optimizer to find the best values to maximize the target value for a and b given the inputs of 3 and 2.

In [7]:
optimizer.maximize(3,2)

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
| 1         | 8.601     | 1.749     | 6.852     |
| 2         | 8.26      | 2.464     | 5.796     |
| 3         | 5.78      | 1.312     | 4.468     |
| 4         | 9.787     | 2.816     | 6.971     |
| 5         | 9.54      | 3.0       | 6.54      |


Great, now let's print the best parameters and the associated maximized target.

In [8]:
print(optimizer.max['params']);optimizer.max['target']

{'a': 2.8160386493536476, 'b': 6.971056555619421}


9.787095204973069

## Test it on real data using the Light GBM

The dataset we will be working with is the famous flight departures dataset. Our modeling goal will be to predict if a flight departure is going to be delayed by 15 minutes based on the other attributes in our dataset. As part of this modeling exercise, we will use Bayesian hyperparameter optimization to identify the best parameters for our model.

**<font color='teal'> You can load the zipped csv files just as you would regular csv files using Pandas read_csv. In the next cell load the train and test data into two seperate dataframes. </font>**


In [9]:
train_df = pd.read_csv('flight_delays_train.csv')
test_df = pd.read_csv('flight_delays_test.csv')

In [38]:
test_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


**<font color='teal'> Print the top five rows of the train dataframe and review the columns in the data. </font>**

In [10]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


**<font color='teal'> Use the describe function to review the numeric columns in the train dataframe. </font>**

In [11]:
train_df.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


Notice, `DepTime` is the departure time in a numeric representation in 2400 hours. 

 **<font color='teal'>The response variable is 'dep_delayed_15min' which is a categorical column, so we need to map the Y for yes and N for no values to 1 and 0. Run the code in the next cell to do this.</font>**

In [12]:
#train_df = train_df[train_df.DepTime <= 2400].copy()
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

## Feature Engineering
Use these defined functions to create additional features for the model. Run the cell to add the functions to your workspace.

 - sklearn [label encoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)
   - encodes labels with 0 to n-1 values
   - consider vs one-hot-encoding 
   - ordinal encoding is the same, but preserves an "order"
 - **trignometric transformers:** sin/cosine harmonic features, see [sklearn time-related feature engineering guide](https://scikit-learn.org/stable/auto_examples/applications/plot_cyclical_feature_engineering.html)
   - smooth time data to its natural periodicity, remove artificial big difference of hours aa day changes
   - see also periodic spline-based encoding, one-hot time encoding (treat hours/days... as categories)

In [13]:
def label_enc(df_column):
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column

def make_harmonic_features_sin(value, period=2400):
    value *= 2 * np.pi / period 
    return np.sin(value)

def make_harmonic_features_cos(value, period=2400):
    value *= 2 * np.pi / period 
    return np.cos(value)

def feature_eng(df):
    df['flight'] = df['Origin']+df['Dest']
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32')
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8')
    df['midddle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')
    df['end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8')
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32')
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['day'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')
    df['spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32')
    df['summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32')
    df['autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32')
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int) 
    df['weekday'] = (df['DayOfWeek'] < 5).astype(int)
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count')
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count')
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count')
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count')
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count')
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count')
    df['deptime_cos'] = df['DepTime'].map(make_harmonic_features_cos)
    df['deptime_sin'] = df['DepTime'].map(make_harmonic_features_sin)
    df['flightUC'] = df['flight']+df['UniqueCarrier']
    df['DestUC'] = df['Dest']+df['UniqueCarrier']
    df['OriginUC'] = df['Origin']+df['UniqueCarrier']
    return df.drop('DepTime', axis=1)

Concatenate the training and testing dataframes.


In [14]:
full_df = pd.concat([train_df.drop('dep_delayed_15min', axis=1), test_df])
full_df = feature_eng(full_df)

Apply the earlier defined feature engineering functions to the full dataframe.

In [15]:
for column in ['UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']:
    full_df[column] = label_enc(full_df[column])


Split the new full dataframe into X_train and X_test. 

In [16]:
X_train = full_df[:train_df.shape[0]]
X_test = full_df[train_df.shape[0]:]

In [ ]:
#train_df = train_df[train_df.DepTime <= 2400].copy()
# y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

Create a list of the categorical features.

In [17]:
categorical_features = ['Month',  'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']

Let's build a light GBM model to test the bayesian optimizer.

### [LightGBM](https://lightgbm.readthedocs.io/en/latest/) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

First, we define the function we want to maximize and that will count cross-validation metrics of lightGBM for our parameters.

Some params such as num_leaves, max_depth, min_child_samples, min_data_in_leaf should be integers.

In [18]:
def lgb_eval(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    params = {
        "objective" : "binary",
        "metric" : "auc", 
        'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1
    }
    lgtrain = lightgbm.Dataset(X_train, y_train,categorical_feature=categorical_features)
    cv_result = lightgbm.cv(params,
                       lgtrain,
                       1000,
                       early_stopping_rounds=100,
                       stratified=True,
                       nfold=3)
    return cv_result['auc-mean'][-1]

Apply the Bayesian optimizer to the function we created in the previous step to identify the best hyperparameters. We will run 10 iterations and set init_points = 2.


In [19]:
%%time

lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (25, 4000), # 31 default, specify as integer. AKA num_leaf, max_leaves, max_leaf
                                                'max_depth': (5, 63), # <=0 is no limit, otherewise specify as integer
                                                'lambda_l2': (0.0, 0.05), # defaults to 0, specify as float
                                                'lambda_l1': (0.0, 0.05), # defaults to 0, specify as float
                                                'min_child_samples': (50, 10000), # minimum data in leaf, use to prevent overfitting. 
                                                'min_data_in_leaf': (100, 2000) # default 20, AKA min_data_per_leaf, min_data. shouldn't be more than child samples?
                                                })

lgbBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
[LightGBM] [Warning] min_data_in_leaf is set=1876, min_child_samples=1792 will be ignored. Current value: min_data_in_leaf=1876
| 1         | 0.7436    | 0.03773   | 0.04726   | 34.22     | 1.792e+03 | 1.877e+03 | 1.803e+03 |
| 2         | 0.7304    | 0.02387   | 0.0002838 | 61.6      | 6.415e+03 | 977.5     | 1.557e+03 |
| 3         | 0.7418    | 0.01204   | 0.009346  | 6.369     | 5.387e+03 | 1.267e+03 | 2.485e+03 |
| 4         | 0.7432    | 0.00271   | 0.01856   | 8.628     | 2.915e+03 | 1.917e+03 | 3.591e+03 |
| 5         | 0.7417    | 0.04304   | 0.008721  | 48.36     | 3.764e+03 | 1.145e+03 | 25.14     |
| 6         | 0.7435    | 0.02404   | 0.03293   | 8.094     | 637.3     | 1.566e+03 | 3.662e+03 |
| 7         | 0.7285    | 0.01922   | 0.03501   | 60.36     | 7.706e+03 | 901.8     | 3.

 **<font color='teal'> Print the best result by using the '.max' function.</font>**

In [26]:
lgbBO.max

{'target': 0.7435800869884351,
 'params': {'lambda_l1': 0.0377307799118757,
  'lambda_l2': 0.04725842065448061,
  'max_depth': 34.21522635903086,
  'min_child_samples': 1792.3108947740304,
  'min_data_in_leaf': 1876.6277775311082,
  'num_leaves': 1802.8288947626093}}

In [31]:
    params = {
        "objective" : "binary",
        "metric" : "auc", 
        'is_unbalance': True,
        "num_leaves" : int(lgbBO.max['params']['num_leaves']),
        "max_depth" : int(lgbBO.max['params']['max_depth']),
        "lambda_l2" : lgbBO.max['params']['lambda_l2'],
        "lambda_l1" : lgbBO.max['params']['lambda_l1'],
        "num_threads" : 20,
        "min_child_samples" : int(lgbBO.max['params']['min_child_samples']),
        'min_data_in_leaf': int(lgbBO.max['params']['min_data_in_leaf']),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1
    }

Review the process at each step by using the '.res[0]' function.

*res[0] is first set of parameters, which were max score. I printed all of them*

In [24]:
lgbBO.res

[{'target': 0.7435800869884351,
  'params': {'lambda_l1': 0.0377307799118757,
   'lambda_l2': 0.04725842065448061,
   'max_depth': 34.21522635903086,
   'min_child_samples': 1792.3108947740304,
   'min_data_in_leaf': 1876.6277775311082,
   'num_leaves': 1802.8288947626093}},
 {'target': 0.7304406297631143,
  'params': {'lambda_l1': 0.023871556372399336,
   'lambda_l2': 0.0002837555734975739,
   'max_depth': 61.596077496423284,
   'min_child_samples': 6415.349238205708,
   'min_data_in_leaf': 977.5330625020368,
   'num_leaves': 1556.7150947465268}},
 {'target': 0.7418381747440236,
  'params': {'lambda_l1': 0.012036003087323095,
   'lambda_l2': 0.00934575716708913,
   'max_depth': 6.368968529577158,
   'min_child_samples': 5387.476000208586,
   'min_data_in_leaf': 1266.6168134735835,
   'num_leaves': 2484.7433754722115}},
 {'target': 0.7432162535327188,
  'params': {'lambda_l1': 0.002709686613156315,
   'lambda_l2': 0.01855780906216123,
   'max_depth': 8.628360281368387,
   'min_child_sa

In [32]:
bst = lightgbm.train(params, lightgbm.Dataset(X_train, y_train,categorical_feature=categorical_features))

In [40]:
y_pred = bst.predict(X_test)

In [41]:
y_pred

array([0.1376741 , 0.19474247, 0.12720465, ..., 0.56360505, 0.29239408,
       0.26304908])

### ???

*will "turn in" notebook and may comeback to do more, updates after submission are below . . .*

- is this all?
- should I try the same thing with CatBoost?
- score/evaluate predictions
  - confusion matrix, make up problem statement and choose metric?
  - test set does not have target feature, because it's for Kaggle submissions?
    - can find Kaggle competition and submit best moodel prdictions
